In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
data = pd.read_csv('/content/Africa towers.csv')

In [ ]:
data.head()

In [ ]:
drop_columns=['unit','changeable','averageSignal','Continent']
data=data.drop(drop_columns,axis=1)

In [ ]:
data.head()

In [ ]:
data['Country'].unique()

In [ ]:
data = data[data['Country'] == 'Egypt']


In [ ]:
# rename column Unnamed: 0
data = data.rename(columns={'Unnamed: 0': ' ID'})

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.columns = data.columns.str.strip()


In [ ]:

data = data[data["Country"].astype(str).str.strip() == "Egypt"].reset_index(drop=True)

# ------------------------------------------------------
# 1. DERIVED KPIs
# ------------------------------------------------------

# Tower Density = number of towers / area
# --> You need area, but we simulate by grouping per city/region later

# Drop Rate (simulate: RANGE column can represent traffic or attempts)
np.random.seed(42)
data["drop_calls"] = np.random.randint(0, 50, len(data))
data["total_calls"] = data["RANGE"] + data["drop_calls"]
data["drop_rate"] = (data["drop_calls"] / data["total_calls"]) * 100

# Average Load per Tower = total_calls / tower count (per MCC/MNC area)
tower_counts = data.groupby(["MCC", "MNC"])["ID"].transform("count")
data["avg_load"] = data["total_calls"] / tower_counts

# QoE Score (0–5 scale) → composite measure
data["signal_strength"] = np.random.randint(-105, -60, len(data))
data["speed"] = np.random.uniform(1, 80, len(data))          # Mbps
data["latency"] = np.random.uniform(10, 200, len(data))      # ms

# Simple QoE formula
data["QoE"] = (
    ((data["signal_strength"] + 110) / 50) * 0.4 +  # normalize 0-1
    (data["speed"] / 80) * 0.4 +
    (1 - (data["latency"] / 200)) * 0.2
) * 5

# Coverage Gap: distance > 2 km
# Simple approximation using RANGE
data["coverage_gap"] = data["RANGE"] > 2000

# ------------------------------------------------------
# 2. CLASSIFICATIONS
# ------------------------------------------------------

# Signal Quality
def classify_signal(dBm):
    if -70 <= dBm <= -60:
        return "Excellent"
    elif -80 <= dBm < -70:
        return "Good"
    elif -90 <= dBm < -80:
        return "Fair"
    elif -100 <= dBm < -90:
        return "Poor"
    else:
        return "Very Poor"

data["signal_quality"] = data["signal_strength"].apply(classify_signal)


# Tower Status
def tower_status(dr):
    if dr < 20: return "Healthy"
    elif dr < 40: return "Warning"
    elif dr < 60: return "Critical"
    else: return "Failed"

data["tower_status"] = data["drop_rate"].apply(tower_status)


# Priority Level
def assign_priority(status):
    if "Failed" in status or "Critical" in status:
        return "P1"
    elif "Warning" in status:
        return "P2"
    else:
        return "P3"

data["priority"] = data["tower_status"].apply(assign_priority)

data.head()

In [ ]:
data.head()

,ID,radio,MCC,MNC,TAC,CID,LON,LAT,RANGE,SAM,...,priority,maintenance_type,created_dt,updated_dt,maintenance_date,labor_cost_egp,parts_cost_egp,downtime_hours,vendor,notes
0,12847759,GSM,602,3,21333,25372,31.056510,29.998215,23897,10,...,P3,predictive,2016-04-03 20:25:36,2016-06-08 01:07:44,2016-05-19 03:32:39.863080001,4294,4318,5.3,ZTE,
1,12847760,GSM,602,3,21362,23224,31.373520,29.839554,1000,4,...,P3,preventive,2016-04-04 23:50:31,2016-09-18 15:36:37,2016-08-08 12:27:08.767132426,3336,1206,2.8,ZTE,
2,12847761,GSM,602,3,22533,5031,31.160660,29.998856,1000,1,...,P3,preventive,2016-04-03 15:05:55,2016-04-03 15:05:55,2016-04-03 15:05:55.000000000,3868,2133,2.3,Ericsson,
3,12847762,GSM,602,3,22202,40686,31.501236,30.592117,1000,1,...,P3,preventive,2016-04-03 11:11:47,2016-04-03 11:11:47,2016-04-03 11:11:47.000000000,2338,1736,2.4,Huawei,Transmission issue
4,12847763,GSM,602,3,21333,25376,31.277390,30.095673,1000,2,...,P3,predictive,2016-04-03 20:25:36,2016-04-12 16:31:42,2016-04-07 22:49:47.089403291,4647,1995,4.7,ZTE,Software upgrade


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53048 entries, 0 to 53047
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                53048 non-null  int64         
 1   radio             53048 non-null  object        
 2   MCC               53048 non-null  int64         
 3   MNC               53048 non-null  int64         
 4   TAC               53048 non-null  int64         
 5   CID               53048 non-null  int64         
 6   LON               53048 non-null  float64       
 7   LAT               53048 non-null  float64       
 8   RANGE             53048 non-null  int64         
 9   SAM               53048 non-null  int64         
 10  created           53048 non-null  int64         
 11  updated           53048 non-null  int64         
 12  Country           53048 non-null  object        
 13  Network           53048 non-null  object        
 14  drop_calls        5304

In [ ]:
data['vendor'].value_counts()

,count
vendor,
FiberMisr,7649
Nokia,7628
Ericsson,7613
Benya,7586
ZTE,7558
Huawei,7528
Local Contractor,7486


In [ ]:
np.random.seed(42)

# 1. Maintenance type
maintenance_types = ["preventive", "predictive", "emergency"]
maintenance_prob = [0.60, 0.25, 0.15]
data["maintenance_type"] = np.random.choice(
    maintenance_types, size=len(data), p=maintenance_prob
)

#2 Convert created & updated to datetime
data["created_dt"] = pd.to_datetime(data["created"], unit="s")
data["updated_dt"] = pd.to_datetime(data["updated"], unit="s")

# Ensure updated >= created (just in case)
mask = data["updated_dt"] < data["created_dt"]
data.loc[mask, "updated_dt"] = data.loc[mask, "created_dt"] + pd.Timedelta(days=30)

# Maintenance type
maintenance_types = ["preventive", "predictive", "emergency"]
maintenance_prob = [0.60, 0.25, 0.15]
data["maintenance_type"] = np.random.choice(
    maintenance_types, size=len(data), p=maintenance_prob
)

def generate_maintenance_date(row):
    start = row["created_dt"]
    end = row["updated_dt"]

    # Total seconds between created and updated
    total_seconds = int((end - start).total_seconds())

    if total_seconds <= 0:
        return start  # fallback

    if row["maintenance_type"] == "preventive":
        # Preventive: close to updated (last 3–6 months)
        offset_seconds = np.random.uniform(total_seconds * 0.6, total_seconds * 1.0)

    elif row["maintenance_type"] == "predictive":
        # Predictive: middle of the lifecycle
        offset_seconds = np.random.uniform(total_seconds * 0.3, total_seconds * 0.7)

    else:  # emergency
        # Emergency: anywhere randomly
        offset_seconds = np.random.uniform(0, total_seconds)

    return start + pd.Timedelta(seconds=offset_seconds)

# Generate the maintenance date
data["maintenance_date"] = data.apply(generate_maintenance_date, axis=1)

# 3. Costs based on maintenance type
def gen_labor_cost(mt):
    if mt == "preventive": return np.random.randint(1500, 4000)
    if mt == "predictive": return np.random.randint(3000, 7000)
    return np.random.randint(5000, 15000)

def gen_parts_cost(mt):
    if mt == "preventive": return np.random.randint(500, 2500)
    if mt == "predictive": return np.random.randint(1000, 5000)
    return np.random.randint(3000, 20000)

data["labor_cost_egp"]  = data["maintenance_type"].apply(gen_labor_cost)
data["parts_cost_egp"] = data["maintenance_type"].apply(gen_parts_cost)

# 4. Downtime hours
def gen_downtime(mt):
    if mt == "preventive": return round(np.random.uniform(0.5, 3), 1)
    if mt == "predictive": return round(np.random.uniform(1, 6), 1)
    return round(np.random.uniform(4, 24), 1)

data["downtime_hours"] = data["maintenance_type"].apply(gen_downtime)

# 5. Vendors
vendors = ["Huawei", "Ericsson", "Nokia", "ZTE", "FiberMisr", "Benya", "Local Contractor"]
data["vendor"] = np.random.choice(vendors, size=len(data))

# 6. Notes
notes_list = [
    "Replaced sector antenna",
    "Battery replaced",
    "Fiber cut repaired",
    "Power system failure",
    "Software upgrade",
    "Transmission issue",
    "Cooling system maintenance",
    ""
]
data["notes"] = np.random.choice(notes_list, size=len(data), p=[0.08,0.08,0.08,0.08,0.08,0.08,0.02,0.5])

In [ ]:
data.head()

,ID,radio,MCC,MNC,TAC,CID,LON,LAT,RANGE,SAM,...,priority,maintenance_type,created_dt,updated_dt,maintenance_date,labor_cost_egp,parts_cost_egp,downtime_hours,vendor,notes
0,12847759,GSM,602,3,21333,25372,31.056510,29.998215,23897,10,...,P3,predictive,2016-04-03 20:25:36,2016-06-08 01:07:44,2016-05-19 03:32:39.863080001,4294,4318,5.3,ZTE,
1,12847760,GSM,602,3,21362,23224,31.373520,29.839554,1000,4,...,P3,preventive,2016-04-04 23:50:31,2016-09-18 15:36:37,2016-08-08 12:27:08.767132426,3336,1206,2.8,ZTE,
2,12847761,GSM,602,3,22533,5031,31.160660,29.998856,1000,1,...,P3,preventive,2016-04-03 15:05:55,2016-04-03 15:05:55,2016-04-03 15:05:55.000000000,3868,2133,2.3,Ericsson,
3,12847762,GSM,602,3,22202,40686,31.501236,30.592117,1000,1,...,P3,preventive,2016-04-03 11:11:47,2016-04-03 11:11:47,2016-04-03 11:11:47.000000000,2338,1736,2.4,Huawei,Transmission issue
4,12847763,GSM,602,3,21333,25376,31.277390,30.095673,1000,2,...,P3,predictive,2016-04-03 20:25:36,2016-04-12 16:31:42,2016-04-07 22:49:47.089403291,4647,1995,4.7,ZTE,Software upgrade


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53048 entries, 0 to 53047
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                53048 non-null  int64         
 1   radio             53048 non-null  object        
 2   MCC               53048 non-null  int64         
 3   MNC               53048 non-null  int64         
 4   TAC               53048 non-null  int64         
 5   CID               53048 non-null  int64         
 6   LON               53048 non-null  float64       
 7   LAT               53048 non-null  float64       
 8   RANGE             53048 non-null  int64         
 9   SAM               53048 non-null  int64         
 10  created           53048 non-null  int64         
 11  updated           53048 non-null  int64         
 12  Country           53048 non-null  object        
 13  Network           53048 non-null  object        
 14  drop_calls        5304

0        False
1        False
2        False
3        False
4         True
         ...  
53043     True
53044     True
53045     True
53046     True
53047     True
Name: created, Length: 53048, dtype: bool


In [ ]:
print(data['created'])
print(data['created_dt'])



In [ ]:
print(data['updated'])
print(data['updated_dt'])

In [ ]:
data['maintenance_date']

In [ ]:
data.to_csv("FULL_telecom_dataset.csv", index=False)


In [ ]:
data.shape

(53048, 35)

In [ ]:
import numpy as np
import pandas as pd
import time

def stream_tower_updates(df, n=5):
    df = df.copy()  # protect original data

    # Select 5 random towers
    sample = df.sample(n).copy()
    now_ts = int(time.time())
    now_dt = pd.Timestamp.now()

    for i in sample.index:

        # --------------------------
        # UPDATE THE "updated" FIELD
        # --------------------------
        sample.loc[i, "updated"] = now_ts
        sample.loc[i, "updated_dt"] = now_dt

        # --------------------------
        # Dynamic KPI Changes
        # --------------------------
        sample.loc[i, "drop_calls"] = max(
            0, sample.loc[i, "drop_calls"] + np.random.randint(-2, 6)
        )
        sample.loc[i, "total_calls"] = max(
            sample.loc[i, "drop_calls"] + 1,
            sample.loc[i, "total_calls"] + np.random.randint(-10, 30)
        )
        sample.loc[i, "drop_rate"] = round(
            sample.loc[i, "drop_calls"] / sample.loc[i, "total_calls"], 4
        )

        # KPI variations
        sample.loc[i, "avg_load"] = round(
            min(1, max(0, sample.loc[i, "avg_load"] + np.random.uniform(-0.05, 0.08))), 3
        )
        sample.loc[i, "signal_strength"] = np.random.randint(-110, -40)
        sample.loc[i, "speed"] = round(np.random.uniform(2, 150), 2)
        sample.loc[i, "latency"] = round(np.random.uniform(10, 150), 2)
        sample.loc[i, "QoE"] = round(
            np.random.uniform(1, 5) - sample.loc[i, "drop_rate"] * 1.5, 2
        )

        # coverage gap
        sample.loc[i, "coverage_gap"] = sample.loc[i, "signal_strength"] < -100

        # signal quality
        if sample.loc[i, "QoE"] > 4:
            sample.loc[i, "signal_quality"] = "Excellent"
        elif sample.loc[i, "QoE"] > 3:
            sample.loc[i, "signal_quality"] = "Good"
        elif sample.loc[i, "QoE"] > 2:
            sample.loc[i, "signal_quality"] = "Fair"
        else:
            sample.loc[i, "signal_quality"] = "Poor"

        # tower status
        sample.loc[i, "tower_status"] = np.random.choice(
            ["active", "warning", "down"], p=[0.85, 0.10, 0.05]
        )

        # priority logic
        if sample.loc[i, "tower_status"] == "down":
            sample.loc[i, "priority"] = "critical"
        elif sample.loc[i, "drop_rate"] > 0.1 or sample.loc[i, "avg_load"] > 0.85:
            sample.loc[i, "priority"] = "high"
        elif sample.loc[i, "drop_rate"] > 0.05:
            sample.loc[i, "priority"] = "medium"
        else:
            sample.loc[i, "priority"] = "low"

        # Maintenance small update
        sample.loc[i, "maintenance_date"] = now_dt - pd.Timedelta(
            days=np.random.randint(1, 120)
        )

        # slight cost variations
        sample.loc[i, "labor_cost_egp"] = int(sample.loc[i, "labor_cost_egp"] * np.random.uniform(0.95, 1.1))
        sample.loc[i, "parts_cost_egp"] = int(sample.loc[i, "parts_cost_egp"] * np.random.uniform(0.9, 1.2))
        sample.loc[i, "downtime_hours"] = round(
            sample.loc[i, "downtime_hours"] * np.random.uniform(0.8, 1.2), 1
        )

        # occasional notes update
        if np.random.random() < 0.08:
            sample.loc[i, "notes"] = np.random.choice([
                "Temporary power fluctuation",
                "Minor transmission delay observed",
                "Environmental interference",
                "Cooling system alert",
                ""
            ])

    return sample


In [ ]:
while True:
    batch = stream_tower_updates(data, n=5)
    print(batch)
    print("\n--------------------------------------\n")
    time.sleep(2)  # stream every 2 seconds


             ID radio  MCC  MNC    TAC       CID        LON        LAT  RANGE  \
35787  12883546  UMTS  602    3  19321    418462  31.302109  30.057907   1000   
35803  12883562  UMTS  602    3  24441    471860  31.277390  29.998856   1000   
40763  12888522  UMTS  602    3  31051    364334  31.353607  30.052414   1000   
35743  12883502  UMTS  602    3  19321    372821  31.330948  30.046921   1000   
9567   12857326  UMTS  602    3  32031  13348560  31.164780  30.949173   1000   

       SAM  ...  priority  maintenance_type          created_dt  \
35787    1  ...       low        preventive 2016-06-29 18:42:19   
35803    2  ...  critical        preventive 2016-06-30 01:07:16   
40763    2  ...       low        predictive 2016-09-28 18:09:11   
35743    2  ...       low        preventive 2016-06-29 03:06:51   
9567     1  ...       low        predictive 2015-06-22 19:43:18   

                      updated_dt           maintenance_date  labor_cost_egp  \
35787 2025-12-09 19:51:37.84810

KeyboardInterrupt: 